# Testbench Creation for Lookup Table Generation

## Introduction


This document shows the process to create a testbench to generate a Lookup Table (LUT) for transistor sizing.
The how and why of this testbench is explained in [Systematic Design of Analog CMOS Circuits](https://doi.org/10.1017/9781108125840).
But the gist of it is that the LUT we are generating here is supposed to capture "all the voltages" and "lengths" a transistor could see in design.
Make a note that we ignore the widths since most of the parameters scale linearly with width or are not affected by width to the first order.

The author of the book has provided a set of scripts as a [starter kit](https://github.com/bmurmann/Book-on-gm-ID-design).
However, upon the author's own admission "The provided scripts are far from 'plug and play'.
You need to look at each line in the header of the configuration file and determine appropriate adjustments for your system."

I have made the author's script work thrice in my life and it took about 5 days every time to get it working for new technology. The first time I had
help but the last two times I was on my own and trust me, you will see stars before you get it working.

However, with the following "manual" approach the setup time can be reduced to less than 30 minutes and does not care how your netlist is generated and models are setup.
Additionally, the biggest contributors to the headache of generating these models were `cds_srr` and the `cds_innersrr` functions. I have completely bypassed this using
simple export from ADE-Assembler to CSV and then parsing this CSV file using Python, however, you can effectively use any language to parse this CSV file and generate a LUT.
Let's get started.

This method might be time consuming for the simulator and my importer scripts are far from efficient, however, they are not as time consuming for the user.


## Schematic Setup

- Setup the testbench exactly as illustrated in the following schematic.
- **Strictly** follow the device, source and net naming conventions to make importing hassle-free.
- Setup the width > 10 um.
- Now if you have multiple device types just copy the devices and rename the new devices.
- For example if you have a SVT, LVT and Native devices you could name them as nmos_svt, nmos_lvt, nmos_native respectively. 
- Please ensure the signs of the voltage expressions are exactly matching.
- Again, don't be a smart-ass and follow the guidelines if you want a hassle-free import.

<figure>
<center>
<img src="../images/testbench_schematic.png" style="width:90%"/>
</center>
<figcaption align = "center"> Testbench Schematic Setup </figcaption>
</figure>

## Maestro View Setup

- Open maestro and create a test named "dc_sweep".
- Add DC analysis and set it up as shown in the image below.


<figure>
<center>
<img src="../images/dc_analysis_setup.png" style="width:50%"/> 
</center>
<figcaption align = "center"> DC Analysis Setup </figcaption>
</figure>

- Add Noise analysis and set it up as shown in the image

<figure>
<center>
<img src="../images/noise_analysis_setup.png" style="width:50%"/> 
</center>
<figcaption align = "center"> Noise Analysis Setup </figcaption>
</figure>

- Now right click on `Design Variables` and click on `Copy from Cellview`. This should copy the variables `ds`, `gs`, `length`, and `sb`. (If you followed the instructions exactly.)
- Set up Linear Sweeps as shown in the image below.
- For the initial test run make the number of sweeps low.
- For the final simulation we will follow a more finer step size for voltages and lengths.
- The table below the image is just a recommendation and you will be the best judge of your resolution based on the technology.

<figure>
<center>
<img src="../images/dc_sweep_setup.png" style="width:50%"/>
</center>
<figcaption align = "center"> DC Sweep Setup </figcaption>
</figure>

| Variable Name             | Start | Stop     | Step Size | Explanation                                                                                                                                                                                                                                                       |
|---------------------------|-------|----------|-----------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| ds (Drain Source Voltage) | 0     | vddmax   | 10m       | For vddmax < 2V use 10 mV step size, for vddmax > 2V use 25 mV step size                                                                                                                                                                                          |
| gs (Gate Source Voltage)  | 0     | vddmax   | 10m       | For vddmax < 2V use 10 mV step size, for vddmax > 2V use 25 mV step size                                                                                                                                                                                          |
| sb (Source Bulk Voltage)  | 0     | vddmax/2 | 50m/100m  | For Bulk CMOS use 50 mV, for FinFETs and others where Bulk Source Voltage hardly matters you can safely make it coarser.  However, even for a coarser step size make sure to cover the entire range from 0 - 0.5*vddmax to capture the deltas due to body effect. At the minimum you need to have atleast two values otherwise the look_up function won't work in mode 3. (It was really painful day when I had to debug this issue so please for the sake of all that is holy simulate for atleast two values of sb.) If you are still adamant about not simulating for two different voltages, which is reasonable under special circumstance see the section **Note regarding sb dimension** [in the import demo document.](./import_demo.ipynb)  |
| length                    | min   | 10u      | Variable  | Finer step sizes (50 nm - 100 nm) till 1um and then 1um step sizes from 1u - 10u. To simulate longer lengths you can use stacking within your scripts but would not recommend blowing up the LUT size to simulate longer lengths here.                            |

### Step Size Example

Say you have the following min and max values for voltages and lengths.

| Variable Name             | Step Size | Start | Stop | Number of Steps. | Explanation                                           |
|---------------------------|-----------|-------|------|------------------|-------------------------------------------------------|
| ds (Drain Source Voltage) | 25m       | 0     | 6    | 240              | for vddmax > 2V use 25 mV step size                   |
| gs (Gate Source Voltage)  | 25m       | 0     | 6    | 240              | for vddmax > 2V use 25 mV step size                   |
| sb (Source Bulk Voltage)  | 50m       | 0     | 3    | 60               | For Bulk CMOS use 50 mV                               |
| length                    | Variable  | 400n  | 25u  | 31               | 100 nm till 1um and then 1um step sizes from 1u - 25u |

Total points = 107 million points.

A smarter way would be the following.
Say the approximate Vt of the devices you want to characterize is 800 mV.
Now realistically speaking, the deepest you would want to bias your device
into weak inversion would be a Vov= -150 mV. This means we do not need
the finer step size till atleast 650 mV (600 mV if we add a margin of 50 mV)
Now for the higher limit of Vov, say you design for a minimum gm/id of 2, that brings
the Vov = 1 thus Vgs = 1.8 V. A similar approach for the drain source voltage
can be followed with additional flexibility beyond the Vds = 1.8 V since,
the parameters of interest won't be significantly affected by the drain source voltage
as the device is in saturation. To clarify, we are not eliminating the other points
but just making sure that we are capturing the data with finer resolution where
it is more sensitive and critical; and coarser step sizes elsewhere. Feel free to report any issues
(such as error between the interpolated values and actual simulation values)
you see with the modified approach.

| Variable Name             | From, To, Step Size                                                |
|---------------------------|--------------------------------------------------------------------|
| ds (Drain Source Voltage) | (0, 500 mV, 100 mV); (600 mV, 1.8 V, 50 mV); (1.9 V, 6 V, 100 mV)  |
| gs (Gate Source Voltage)  | (0, 500 mV, 100 mV); (600 mV, 1.8 V, 50 mV); (1.9 V, 6 V, 200 mV)  |
| sb (Source Bulk Voltage)  | (0, 3, 3)                                                          |
| length                    | (0.4 um, 0.95 um, 50 nm); (1 um, 4 um, 1 um); (5 um, 25 um, 5 um)  |

Total points ~ 90,000 points.
However, reduction by a factor of 1000, I see this as an absolute win!

- Edit the `expressions_template_to_import.csv` to match your device naming conventions.
- Please follow the `operatingparameter_devicename` convention strictly for any new expressions added. For example, `cgd` would look like `cgd_nmos`.
- Feel free to add any new operating parameters, but these are the bare minimum parameters required.
- Now import the `expressions_template_to_import.csv` into your current Assembler view.
  - [Video Instructions](https://community.cadence.com/cadence_blogs_8/b/cic/posts/virtuoso-video-diary-reusing-variables-through-csv-export-and-import).
  - To do this, Click on `Outputs Setup Tab`. Then click on icon that looks like `Open Folder` (If you hover your mouse pointer over it will show `Import Outputs from CSV file`). This will open a form where you can Click on `Browse` button and choose your CSV file. For the operation select `Overwrite`.
- Make sure to add your corner or leave it at Nominal. Instructions on how to import this data will be provided. But after the simulation is over export the data generated using these [instructions](https://community.cadence.com/cadence_technology_forums/f/custom-ic-design/58223/export-save-results-data-to-csv-in-explorer)

## Known Issues (GUI Lag)

Have a long ass simulation and don't want it slowing your GUI? Here's the way to run your simulation purely using command line.
However, make sure that you have enabled the relevant tests and corners before doing it!
All the commands are formatted `thisisacommand`.

Set up your environment as you would do before running `virtuoso &` or `icfb &`.

- Run `virtuoso -nograph`. This command will open the virtuoso in purely CIW mode within your terminal.
- Once the CIW is loaded, we will open our maestro setup. To do this you need to exactly specify the library name, cell name and view name.
- For example, if your library is *design_lna*, your cell name is *lna_core_sim* and your view name is *maestro* you should use `maeOpenSetup("design_lna" "lna_core_sim" "maestro")`
- Once you run it you will get something like *fnxSession0* or similar. This is the session name.
- Then we just run the simulation using `maeRunSimulation()`.
- Once you run this you will get something like "Interactive.0".
- Make sure that before you follow this method, to save your maestro in a state with correct tests and corners enabled. This command just blindly simulates what is enabled.
- Go have a coffee/tea and forget about your long-ass simulations blocking your GUI.
- Check up on it every once in a while. If in the terminal it shows something like `"Interactive.0" has finished running` you are done, otherwise it is incomplete.
- When all of it is over, let us export the data to a CSV file. In order to do so, you must know the history name and the session name. In our case, session name is *fnxSession0* and history name is *Interactive.0*. To export *a.csv* to */home/Downloads/* run the following command.
- `maeExportOutputView(?session "fnxSession0" ?fileName "/home/Downloads/a.csv" ?view "Detail" ?historyName "Interactive.0")`

If you have followed the above schematic exactly, you are good to go, however if you would still like to be extra cautious you can compare your netlist generated with netlist [here](https://github.com/bmurmann/Book-on-gm-ID-design/blob/main/starter_kit/techsweep_config_psp_65_spectre.m)


Thank you.

### Acknowlegments

This testbench and accompanying setup and images were generated thanks to precious help of Bharadwaj Padmanabhan.